# Description

This notebook focuses on visualizing nodes within the Pywr-DRB model along with USGS gauge locations near those nodes.

In [1]:
import numpy as np
import pandas as pd
import folium

# Data Preparation

Load the model nodes and edges from the `/model_data` folder. 

Create GeoDataFrame objects for each model component. These components include:

- Nodes:
    - Reservoirs (type = 'storage')
    - River gauges (type = 'rivergauge')
    - Model outputs (type = 'output')
    - Relevant USGS gauges 

- Edges:
    - Mainstem 
    - Tributaries
    - Diversions


In [27]:
### Load nodes, edges, and usgs gauge data
nodes = pd.read_csv('../model_data/drb_model_nodes.csv', sep = ',')
edges = pd.read_csv('../model_data/drb_model_edges.csv', sep = ',')

# Reservoir data
reservoir_data = pd.read_csv('../model_data/drb_model_istarf_conus.csv', sep = ',')


### TODO: Need to get the long,lat data for gauges of interest. 
usgs_gauges = pd.read_csv('../model_data/drb_model_usgs_data_sources.csv', sep = ',')


In [28]:
# Specify node and edge types to be plotted
plot_node_types = ['storage', 'rivergauge', 'output', 'catchment', 'link']
plot_edge_types = ['mainstem', 'tributary', 'diversion']


# Filter for just types of interest
nodes = nodes[nodes['type'].isin(plot_node_types)]
edges = edges[edges['type'].isin(plot_edge_types)]

# Counts
n_nodes = nodes.shape[0]
n_edges = edges.shape[0]

In [29]:
### Add descriptions to nodes of interest
nodes['description'] = ""

## Cannonsville
index = nodes[nodes.name == 'reservoir_cannonsville'].index[0]
nodes.at[index, 'description'] = 'Cannonsville Reservoir <br> One of three major NYC reservoirs.'

### Pepacton
index = nodes[nodes.name == 'reservoir_pepacton'].index[0]
nodes.at[index, 'description'] = 'Pepacton Reservoir <br> One of three major NYC reservoirs.'

### Neversink
index = nodes[nodes.name == 'reservoir_neversink'].index[0]
nodes.at[index, 'description'] = 'Neversink Reservoir <br> One of three major NYC reservoirs.'


### Blue marsh
index = nodes[nodes.name == 'reservoir_blueMarsh'].index[0]
nodes.at[index, 'description'] = 'Blue Marsh Reservoir <br> A flood control reservoir operated by ACE.'

### Trenton site
index = nodes[nodes.name == 'outflow_trenton'].index[0]
nodes.at[index, 'description'] = 'Trenton Gauge Site <br> Point of interest for water policy.'


# Geo-plotting

The `folium` package is used. 

See the documentation here: https://python-visualization.github.io/folium/

In [76]:
# Initialize the map
start_coords = [40.7, -75]
geomap = folium.Map(location = start_coords, zoom_start = 8)
folium.TileLayer('cartodbpositron').add_to(geomap)

geomap.save('path')

In [77]:
# Filter edges which do not have corresponding nodes
edges = edges[edges['node1'].isin(nodes.name)]
edges = edges[edges['node2'].isin(nodes.name)]

# Link colors and dimensions
tributary_color = '#9fc5e8'
mainstem_color = '#397aa7'
diversion_color = '#f0b237'

tributary_weight = 3
mainstem_weight = 7
diversion_weight = 5

edge_opacity = 0.8

# Add links
for l in range(n_edges):
    coord_1 = [nodes[nodes["name"] == edges["node1"][l]].lat.iloc[0], nodes[nodes["name"] == edges["node1"][l]].long.iloc[0]]
    coord_2 = [nodes[nodes["name"] == edges["node2"][l]].lat.iloc[0], nodes[nodes["name"] == edges["node2"][l]].long.iloc[0]]
    line = [coord_1, coord_2]
    
    edge_type = edges.type.iloc[l]
    
    if edge_type == 'mainstem':
        folium.PolyLine(line, 
                       weight = mainstem_weight,
                       color = mainstem_color,
                       opacity = edge_opacity).add_to(geomap)
    elif edge_type == 'tributary':
        folium.PolyLine(line, 
                       weight = tributary_weight,
                       color = tributary_color,
                       opacity = edge_opacity).add_to(geomap)
    elif edge_type == 'diversion':
        folium.PolyLine(line, 
                       weight = diversion_weight,
                       color = diversion_color,
                       opacity = edge_opacity).add_to(geomap)

In [80]:
# Specify colors [marker edges, fill]
storage_colors = ["#3186cc", "#3186cc"]
gauge_colors = ['#78b72c', '#78b72c']
output_colors = ['#b72c78', '#b72c78']
catchment_colors = ["#3186cc", "#3186cc"]
link_colors = ['#5e5e5e', '#5e5e5e']

# Fill opacity
fop = 0.5

popup_width = 300

output_size  = 15
storage_size = 10
catchment_size = 30
link_size = 2

volume_scale = 1/8
max_radius = 50

#icon_name = './icons/reservoir_icon.png'
#reservoir_icon = folium.features.CustomIcon(icon_name, icon_size=(20, 20))

# Add nodes
for n in range(n_nodes):
    coords = [nodes.lat.iloc[n], nodes.long.iloc[n]]
    
    disp = nodes['description'].iloc[n]
    if len(disp) == 0:
        disp = nodes.name.iloc[n]
    
    node_type = nodes.type.iloc[n]
    
    pop = folium.Popup(disp, min_width = popup_width, max_width = popup_width)
        
    if node_type == 'storage':
        
        res_name = nodes.name.iloc[n].split('_')[1]
        volume = reservoir_data[reservoir_data['reservoir'] == res_name]['GRanD_CAP_MCM'].iloc[0]
        if (np.isnan(volume)) or (volume*volume_scale < storage_size):
            s = storage_size
        elif (volume*volume_scale > max_radius):
            s = max_radius
        else:
            s = volume*volume_scale
            
        pop = folium.Popup(f'{disp} <br> Capacity: {volume} MCM', min_width = popup_width, max_width = popup_width)
        
        #res = folium.Marker(coords, icon=reservoir_icon, popup = pop)
        #geomap.add_child(res)

        folium.RegularPolygonMarker(coords,
                                   popup = pop,
                                   number_of_sides = 3,
                                   radius = s,
                                   fill_color = storage_colors[1],
                                   fill_opacity = fop,
                                   color = storage_colors[0],
                                   rotation = 90).add_to(geomap)
        
    elif node_type == 'rivergauge':
        gauge_name = nodes.name.iloc[n]
        if  gauge_name in ['outflow_neversink', 'outflow_cannonsville', 'outflow_pepacton']:
            pass
        else:
            folium.Marker(coords, 
                      popup = pop,
                      icon=folium.Icon(color="green")).add_to(geomap)
        
    elif node_type == 'output':
        folium.RegularPolygonMarker(coords,
                                   popup = pop,
                                   number_of_sides = 4, 
                                   radius = output_size,
                                   fill_color= output_colors[1],
                                    fill_opacity = fop,
                                   color = output_colors[0]).add_to(geomap)
        
    elif node_type == 'catchment':
        pass
        #folium.CircleMarker(coords, 
         #                   popup = pop,
          #                  fill_color = catchment_colors[1],
           #                 fill = True,
            #                fill_opacity = fop,
             #               color = catchment_colors[0]).add_to(geomap)
    elif node_type == 'link':
        folium.CircleMarker(coords, 
                            popup = pop,
                            fill_color = link_colors[1],
                            fill = False,
                            fill_opacity = 0.2,
                            color = link_colors[0],
                           radius = link_size).add_to(geomap)

In [81]:
# Display the map
geomap

In [216]:
# Save as an HTML
geomap.save("drb_model_map.html")